In [85]:
import pandas as pd
from pymongo import MongoClient

In [86]:
files = [
    '../datasets/주식거래 수수료_1억원.xls'
    ,'../datasets/주식거래 수수료_10만원.xls'
    ,'../datasets/주식거래 수수료_50만원.xls'
    ,'../datasets/주식거래 수수료_100만원.xls'
    ,'../datasets/주식거래 수수료_500만원.xls'
    ,'../datasets/주식거래 수수료_1000만원.xls'
    ]

merged_df = pd.DataFrame()

for file in files:
    df = pd.read_excel(file, header=2)
    merged_df = pd.concat([merged_df,df])

merged_df

,회사명,수수료\n부과기준,기준일자,거래금액,구분,오프라인,HTS,ARS,스마트폰,오프라인.1,HTS.1,ARS.1,스마트폰.1,비고
0,유진투자증권,/img/sub/ico_xls.gif,2012/10/08,1억원,변경전,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15000.0,NaN
1,유진투자증권,/img/sub/ico_xls.gif,2020/06/17,1억원,변경후,500000.00000,100000.0,200000.0,132000.0,500000.0,15000.0,200000.0,15000.0,NaN
2,유진투자증권,/img/sub/ico_xls.gif,2020/06/17,1억원,변경율,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
3,SK증권,/img/sub/ico_xls.gif,2011/11/18,1억원,변경전,500000.00000,120000.0,250000.0,50000.0,500000.0,15000.0,250000.0,50000.0,NaN
4,SK증권,/img/sub/ico_xls.gif,2015/06/01,1억원,변경후,500000.00000,120000.0,250000.0,120000.0,500000.0,15000.0,250000.0,120000.0,증권통App 수수료율 변경
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,비엔피파리바증권,/img/sub/ico_pdf.gif,2020/02/03,1000만원,변경율,-0.23077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,토스증권,/img/sub/ico_pdf.gif,2021/01/21,1000만원,변경후,NaN,NaN,1500.0,1500.0,NaN,NaN,NaN,NaN,NaN
95,비엔케이투자증권,/img/sub/ico_pdf.gif,2016/12/05,1000만원,변경전,NaN,NaN,NaN,15000.0,NaN,NaN,NaN,15000.0,NaN
96,비엔케이투자증권,/img/sub/ico_pdf.gif,2020/08/24,1000만원,변경후,44916.00000,9916.0,NaN,15000.0,44916.0,1416.0,NaN,15000.0,NaN


In [87]:
client = MongoClient('mongodb://python_jupyternote_mongo-mongodb-1:27017')
db_name = client['pandas_db']
stock_tradingfees = db_name['stock_list']

In [88]:
result = merged_df.to_dict(orient = 'records')
stock_tradingfees.insert_many(result)

InsertManyResult([ObjectId('67612ffc4f59cb75635c996a'), ObjectId('67612ffc4f59cb75635c996b'), ObjectId('67612ffc4f59cb75635c996c'), ObjectId('67612ffc4f59cb75635c996d'), ObjectId('67612ffc4f59cb75635c996e'), ObjectId('67612ffc4f59cb75635c996f'), ObjectId('67612ffc4f59cb75635c9970'), ObjectId('67612ffc4f59cb75635c9971'), ObjectId('67612ffc4f59cb75635c9972'), ObjectId('67612ffc4f59cb75635c9973'), ObjectId('67612ffc4f59cb75635c9974'), ObjectId('67612ffc4f59cb75635c9975'), ObjectId('67612ffc4f59cb75635c9976'), ObjectId('67612ffc4f59cb75635c9977'), ObjectId('67612ffc4f59cb75635c9978'), ObjectId('67612ffc4f59cb75635c9979'), ObjectId('67612ffc4f59cb75635c997a'), ObjectId('67612ffc4f59cb75635c997b'), ObjectId('67612ffc4f59cb75635c997c'), ObjectId('67612ffc4f59cb75635c997d'), ObjectId('67612ffc4f59cb75635c997e'), ObjectId('67612ffc4f59cb75635c997f'), ObjectId('67612ffc4f59cb75635c9980'), ObjectId('67612ffc4f59cb75635c9981'), ObjectId('67612ffc4f59cb75635c9982'), ObjectId('67612ffc4f59cb75635c99

In [89]:
filtered_stock = list(stock_tradingfees.find({'구분' : '변경후'}))
filtered_stock_frame = pd.DataFrame(filtered_stock)

In [90]:
filtered_stock_phone = filtered_stock_frame['스마트폰.1'].mean()
filtered_stock_phone

17731.059210526317

In [91]:
filtered_stock_frame_groupby = filtered_stock_frame.groupby('회사명')['스마트폰.1'].mean()

In [92]:
average_above = filtered_stock_frame_groupby[filtered_stock_frame_groupby >= filtered_stock_phone]
average_above

회사명
DB금융투자      20904.166667
IBK투자증권     19433.333333
KB증권        23320.000000
NH투자증권      38565.500000
SK증권        24483.333333
교보증권        29241.666667
대신증권        38341.166667
메리츠증권       19286.666667
부국증권        29066.666667
비엔케이투자증권    29150.000000
삼성증권        21250.333333
상상인증권       27206.666667
신영증권        19270.833333
아이엠증권       19389.166667
유안타증권       19433.333333
한양증권        19433.333333
한화투자증권      29150.000000
Name: 스마트폰.1, dtype: float64

In [93]:
average_above.to_csv('../datasets/Compare_stocktradingfees_merges.csv')